# **Informasi Data**
Dataset yang digunakan untuk membuat model prediksi ini diambil dari situs kaggle, Kaggle adalah situs web yang menyediakan beragam dataset: https://www.kaggle.com/datasets/itsnahm/baldness-probability

dataset ini mencakup beragam aspek yang dapat memengaruhi kemungkinan kebotakan, sehingga memungkinkan untuk mengembangkan model prediksi yang lebih akurat dan informatif.

# #1 Meng-import Library dan cek data

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# cek data
data = pd.read_csv('bald_probability.csv')

data.head()

,age,gender,job_role,province,salary,is_married,is_hereditary,weight,height,shampoo,is_smoker,education,stress,bald_prob
0,27.0,female,Government Employee,Bengkulu,7.957453e+06,1.0,0.0,54.315053,170.428542,Pantone,1.0,Bachelor Degree,5.0,0.605974
1,53.0,female,Government Employee,Bandung,7.633003e+06,1.0,0.0,72.873404,165.530097,Pantone,0.0,Bachelor Degree,7.0,0.532860
2,37.0,female,Employee,Bandung,6.637625e+06,1.0,0.0,46.321533,154.599388,Moonsilk,0.0,Bachelor Degree,4.0,0.418442
3,36.0,female,Jobless,Palu,3.624871e+06,1.0,0.0,51.539781,167.340481,Deadbuoy,1.0,Elementary School,9.0,0.804050
4,38.0,male,NaN,Palangkaraya,6.031808e+06,1.0,0.0,60.726909,165.514773,Merpati,1.0,Magister Degree,1.0,0.368371


# #2. Proses transformasi data
menghapus data yang tidak diperlukan untuk mengoptimalkan dataset. Selanjutnya, melakukan konversi nilai-nilai float ke tipe data integer agar data lebih sesuai dengan kebutuhan analisis. Terakhir, encoding pada string atau kategori dalam dataset untuk mengubahnya menjadi kode numerik

In [ ]:
def transform(data):
  hapus = ['job_role', 'province', 'salary', 'weight', 'height', 'education']
  konvfill = ['age','is_married','is_hereditary','is_smoker','stress','bald_prob']
  Sfill = ['gender','shampoo']

  # Hapus semua data di dalam list
  data.drop(columns=hapus, inplace=True)
  data['bald_prob'] = data['bald_prob']*100

  #Konversi dan fill data Nan menjadi 0
  for i in konvfill:
    data[i].fillna(0, inplace=True)
    data[i] = data[i].astype(int)
  for i in Sfill:
    data[i].fillna('Tidak ada', inplace=True)

  # Lakukan label encoding untuk fitur kategorikal seperti 'gender' dan 'shampoo'
  label_encoder = LabelEncoder()
  data['gender'] = label_encoder.fit_transform(data['gender'])
  data['shampoo'] = label_encoder.fit_transform(data['shampoo'])
  #data['bald_prob'] = label_encoder.fit_transform(data['blad_prob'])

  #Kategori 1: Risiko Kebotakan Rendah (40 <= bald_prob < 60)
  #Kategori 2: Risiko Kebotakan Sedang (60 <= bald_prob < 80)
  #Kategori 3: Risiko Kebotakan Tinggi (80 <= bald_prob)
  data['bald_prob'] = data['bald_prob'].apply(lambda x: 1 if x < 60 else (2 if x < 80 else 3))

#SOLVE DISINI

  return data

trans = transform(data)
datakosong = trans.isna().sum()
print("Jumlah Data yang kosong: \n",datakosong)


Jumlah Data yang kosong: 
 age              0
gender           0
is_married       0
is_hereditary    0
shampoo          0
is_smoker        0
stress           0
bald_prob        0
dtype: int64


In [ ]:
trans.head(5)

,age,gender,is_married,is_hereditary,shampoo,is_smoker,stress,bald_prob
0,27,1,1,0,3,1,5,2
1,53,1,1,0,3,0,7,1
2,37,1,1,0,2,0,4,1
3,36,1,1,0,0,1,9,3
4,38,2,1,0,1,1,1,0


In [ ]:
# Load data ke CSV
trans.to_csv('FinaldataBersih.csv',index=False)

# #3. Mengecek Kategori Encoding untuk kolom Shampoo
hal ini dilakukan untuk mengetahui tiap kode dari jenis shampoo diencoding, berbeda dengan gender yang hanya 2 baris saja yaitu female:1 dan male:2

In [ ]:
def ListEncoderSham(data):
  hapus = ['gender','job_role', 'province', 'salary', 'weight', 'height', 'education','age','is_married','is_hereditary','is_smoker','stress','bald_prob']
  data.drop(columns=hapus,inplace=True)
  data['shampoo'].fillna('Tidak ada', inplace=True)

  # Lakukan label encoding untuk fitur kategorikal 'shampoo'
  label_encoder = LabelEncoder()
  data['Kode_shampoo'] = label_encoder.fit_transform(data['shampoo'])

  # Hapus baris duplikat berdasarkan seluruh kolom dan hanya menampilkan 1 baris yang ada
  data.drop_duplicates(inplace=True)

  return data

enco = ListEncoderSham(data)
datakosong = enco.isna().sum()
print("Jumlah Data yang kosong: \n",datakosong)


Jumlah Data yang kosong: 
 shampoo         0
Kode_shampoo    0
dtype: int64


In [ ]:
enco.head(1000)

,shampoo,Kode_shampoo
0,Pantone,3
2,Moonsilk,2
3,Deadbuoy,0
4,Merpati,1
6,Shoulder & Head,4
105,Tidak ada,5


# #4. Membuat model untuk prediksi probabilitas kebotakan menggunakan algoritma Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Membaca file CSV
data = pd.read_csv('FinaldataBersih.csv')

# Memisahkan fitur (features) dan label (target)
X = data[['age','is_married','is_hereditary','is_smoker','stress','gender','shampoo']]
y = data['bald_prob']

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model Support Vector Machine (SVM)
svm_model = SVC(kernel='linear', random_state=42)

# Melatih model dengan data latih
svm_model.fit(X_train, y_train)

# Melakukan prediksi dengan data uji
y_pred = svm_model.predict(X_test)

# Mengukur kinerja model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Akurasi:", accuracy)
#print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Akurasi: 0.7045454545454546

Classification Report:
               precision    recall  f1-score   support

           1       0.77      0.87      0.82       906
           2       0.56      0.58      0.57       513
           3       0.78      0.18      0.29       165

    accuracy                           0.70      1584
   macro avg       0.71      0.54      0.56      1584
weighted avg       0.71      0.70      0.68      1584



In [ ]:
new_data = pd.DataFrame({'age': [23], # Berapa umur anda sekarang? jawab sesuai angka umur
                     'is_married': [0], # apakah sudah menikah? 0 = belum, 1 = sudah menikah
                     'is_hereditary': [1], # apakah ada faktor genetik dari keturunan ayah/ibu? 0 = tidak ada, 1 = ada
                     'is_smoker': [1], # apakah anda merokok? 0 = tidak, 1 = ya
                     'stress': [10], # seberapa stress anda? beri nilai 1 sampai 10
                     'gender': [2], # jenis kelamin anda? 1 = perempuan, 2 = laki-laki
                     'shampoo': [0]}) # merek sampo yang dipakai? Pantone =	3
                                      # Moonsilk = 2
                                      # Deadbuoy = 0
                                      # Merpati = 1
                                      # Shoulder & Head =	4
                                      # Tidak ada = 5
predicted_category = svm_model.predict(new_data)
print("\nPrediksi Level Kebotakan:", predicted_category[0])
if predicted_category[0] == 1:
  print("Resiko anda mengalami Kebotakan, Rendah!")
elif predicted_category[0] == 2:
  print("Resiko anda mengalami Kebotakan, Sedang!")
else:
  print("Resiko anda mengalami Kebotakan, Tinggi!")


Prediksi Kebotakan untuk Data Baru: 3
Resiko anda mengalami Kebotakan, Tinggi!


**Selanjutnya, saya akan menerapkan model ini menjadi sebuah web application yang dapat diakses secara online, memungkinkan pengguna untuk dengan mudah memasukkan data mereka sendiri dan mendapatkan prediksi probabilitas kebotakan berdasarkan variabel-variabel yang relevan. Dengan demikian, aplikasi web ini akan memberikan akses yang lebih luas dan mudah bagi orang-orang untuk memahami risiko kebotakan mereka.**

Berikut link web aplikasi yang sudah dibuat: http://portofoliofathur.pythonanywhere.com/